# KorGPT2 챗봇

In [1]:
!pip install transformers

## Tokenizer 선언

In [2]:
from transformers import PreTrainedTokenizerFast

Q_TKN = "<user>"
A_TKN = "<sys>"
BOS = "</s>"
EOS = "</s>"
MASK = "<unused0>"
SENT = "<unused1>"
PAD = "<pad>"

In [3]:
# 허깅페이스 transformers에 등록된 사전 학습된 koGTP2 토크나이저를 가져온다.
# PreTrainedTokenizerFast 클래스의 from_pretrained 메소드를 사용하여 사전 훈련된 토크나이저를 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token=BOS,
                                                    eos_token=EOS,
                                                    unk_token="<unk>",
                                                    pad_token=PAD,
                                                    mask_token=MASK,
                                                    )
tokenizer.tokenize("한국어 GPT-2를 활용해보겠습니다.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁한국어', '▁G', 'P', 'T', '-2', '를', '▁활용', '해보', '겠', '습니다.']

## 사전학습된 KoGPT2 언어 모델 객체를 생성

In [4]:
import torch
from transformers import GPT2LMHeadModel # GPT2 LM Head Model : LM Head가 추가된 GPT-2 모델, 주로 자연어 생성 작업에 사용

# GPT2LMHeadModel 클래스의 from_pretrained 메소드를 사용하여 사전 훈련된 GPT-2 모델을 로드
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

text = '딥러닝 개발을 잘 하기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
# encode() : token string을 token id 의 리스트로 변환
# return_tensors : 토큰화된 결과를 파이썬 정수 목록 대신 텐서로 반환
# 'tf': TensorFlow tf.constant 객체
# 'pt': PyTorch torch.Tensor 객체
# 'np': Numpy np.ndarray 객체

gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,   # eos -> end of sentence
                           bos_token_id=tokenizer.bos_token_id,   # bos -> begin of sentence
                           use_cache=True)

# decode() : tokenizer 와 vocabulary를 이용해서 token id를 string으로 변환
generated = tokenizer.decode(gen_ids[0])
print(generated)

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

딥러닝 개발을 잘 하기 위해서는 학습자의 뇌가 학습을 통해 어떤 정보를 얻어야 하는지 알아야 한다.
학습자는 자신의 두뇌를 어떻게 활용하느냐에 따라 그 정보가 얼마나 중요한지를 알 수 있다.
이런 점에서 볼 때 '뇌의 정보'는 매우 중요하다.
'정보'를 얻기 위해선 먼저 뇌의 기능을 이해하고 이를 바탕으로 한 훈련과 훈련을 해야 한다는 것이다.
그렇다면 이 두 가지 측면에서 보면 알파고는 과연 어느 정도의 수준의 지능인지 가늠할 수가 있을 것 같다.
알파고가 개발한 알고리즘은 다음과 같은 특징을 가지고 있는데, 우선, AI와 빅데이터 분석에 필요한 핵심 요소인 데이터 수집 능력을 향상시켰다.
AI는 데이터를 수집하고 분석하는 과정에서


## Chatbot Dataset 다운로드하기
- dataset =  https://github.com/songys/Chatbot_data


In [5]:
import pandas as pd
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
Chatbot_Data = pd.read_csv("ChatBotData.csv")
# print(Chatbot_Data.shape)

Chatbot_Data = Chatbot_Data[:1000]
Chatbot_Data.head()

# Q : 발화자
# A : 발화자
# label : 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


## Dataset 시작하기

In [6]:
from torch.utils.data import DataLoader, Dataset
import numpy as np
import re

# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋 전처리
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        # #질문의 길이가 최대길이보다 크면
        # if q_len > self.max_len:
        #     a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
        #     if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
        #         q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
        #         q_len = len(q_toked)
        #         a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
        #     a_toked = a_toked[:a_len]
        #     a_len = len(a_toked)

        # #질문의 길이 + 답변의 길이가 최대길이보다 크면
        # if q_len + a_len > self.max_len:
        #     a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
        #     if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
        #         q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
        #         q_len = len(q_toked)
        #         a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
        #     a_toked = a_toked[:a_len]
        #     a_len = len(a_toked)

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # 답변 labels을 index 로 만든다 (고유 정수 인코딩)
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)

        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)

        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [7]:
# Helper Function - batchwise
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## 모델 학습하기

In [9]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [10]:
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [11]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18
losses  = 0

train_set = ChatbotDataset(Chatbot_Data, max_len=40)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

print ("start")
for epoch in range(epoch):
    losses  = 0
    # train_dataloader에서 배치 단위로 샘플을 가져와 학습
    for batch_idx, samples in enumerate(train_dataloader):
        # Gradient 0으로 초기화
        optimizer.zero_grad()

        # 샘플에서 token_ids, mask, label을 가져옴
        token_ids, mask, label = samples
        token_ids = token_ids.to(device)
        mask = mask.to(device)
        label = label.to(device)

        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        # print(out.shape)
        # print(mask.shape)
        # print(mask.unsqueeze(dim=2).shape)

        # repeat_interleave : out의 2번째 차원의 크기만큼 반복(repeats=out.shape[2]), 2번째(dim=2) 차원을 따라 반복이 수행
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        # print(mask_3d.shape)

        # mask_3d가 1인 위치에는 출력 값을 그대로 사용,
        # mask_3d가 0인 위치에는 out 배열과 동일한 크기의 배열을 생성하고, 모든 원소의 값을 Sneg 값으로 설정
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))

        # mask_out 배열과 label 사용하여 손실 함수를 계산
        loss = criterion(mask_out.transpose(2, 1), label)

        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()

        # Backward pass (gradient 계산)
        avg_loss.backward()

        # Parameter update
        optimizer.step()

         # 손실 누적
        losses += avg_loss.item()
    print(f'epoch : %5d | loss : %.5f ' %(epoch+1, losses / len(list(train_dataloader))))
print ("end")

start


<ipython-input-7-ec9e45217071>:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


epoch :     1 | loss : 35.78361 
epoch :     2 | loss : 34.46297 
epoch :     3 | loss : 33.91470 
epoch :     4 | loss : 33.65536 
epoch :     5 | loss : 33.32250 
epoch :     6 | loss : 33.03401 
epoch :     7 | loss : 33.11810 
epoch :     8 | loss : 33.07712 
epoch :     9 | loss : 33.00869 
epoch :    10 | loss : 33.01651 
end


## 모델 테스트하기

In [12]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0).to(device)
            pred = model(input_ids)
            pred = pred.logits.to(device)
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
            # PyTorch 텐서는 GPU 메모리에 저장될 수 있지만, NumPy 배열은 항상 호스트(CPU) 메모리에 저장된다.
            # 따라서, GPU 메모리에 저장된 PyTorch 텐서를 직접 NumPy 배열로 변환할 수 없다
            # numpy 메소드를 사용하려면 다음과 같이 처리해줘야한다
            # tensor = tensor.cpu().numpy()
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

user > 안녕
Chatbot > 괜찮은 선택이길 바라요


KeyboardInterrupt: Interrupted by user

## BERT로 네이버 영화 리뷰 데이터 분류하기
- BERT (Bidirectional Encoder Representations from Transformers)
  - 트랜스포머를 기반으로 구현됨
  - 레이블이 없는 텍스트 데이터로 사전 훈련된 언어 모델
  - 트랜스포머의 인코더, 즉 언어 모델링 단계를 쌓아올린 구조


In [13]:
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [14]:
import pandas as pd
import numpy as np
import random
import time
import datetime

In [15]:
## GPU 확인
import os

n_devices = torch.cuda.device_count()
print('device count:',n_devices)

device count: 1


In [17]:
for i in range(n_devices):
    print(i,':',torch.cuda.get_device_name(i))

0 : Tesla T4


In [16]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Counting objects: 100% (14762/14762), done.
remote: Compressing objects: 100% (13012/13012), done.
remote: Total 14763 (delta 1748), reused 14762 (delta 1748), pack-reused 1
Receiving objects: 100% (14763/14763), 56.19 MiB | 16.04 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (14737/14737), done.


In [18]:
## main 및 test 로드
train = pd.read_csv("./nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("./nsmc/ratings_test.txt", sep='\t')

print('train shape:',train.shape)
print('test shape:',test.shape)

train shape: (150000, 3)
test shape: (50000, 3)


### 데이터 전처리
- 문장별 전처리
  - BERT 분류모델은 각 문장의 앞마다 [CLS]를 붙여 인식함
    - CLS -> Special Classification Token
  - 문장 종료는 [SEP]으로 알림
    - SEP -> Special Seperator Token

In [19]:
document_bert = ["[CLS] " + str(s) + " [SEP]" for s in train.document]
document_bert[:5]

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]']

### 토크나이징

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]
print(tokenized_texts[0])

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

['[CLS]', '아', '더', '##빙', '.', '.', '진', '##짜', '짜', '##증', '##나', '##네', '##요', '목', '##소', '##리', '[SEP]']


### 패딩
- token들의 max length보다 크게 MAX_LEN을 설정
- 설정한 MAX_LEN 만큼 빈 공간을 0이 채운다.

In [21]:
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]

array([   101,   9519,   9074, 119005,    119,    119,   9708, 119235,
         9715, 119230,  16439,  77884,  48549,   9284,  22333,  12692,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

### Attention Mask
- 학습속도를 높이기 위해 실 데이터가 있는 곳과 padding이 있는 곳을 attention에게 알려줌

In [22]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


### train-validation set 분리
- input과 mask가 뒤섞이지 않도록 random_state를 일정하게 고정
- test set은 위에서 이미 분리되었기에 train과 validation set만을 분리
- random_state는 42가 일반적

In [24]:
train_inputs, validation_inputs, train_labels, validation_labels = \
train_test_split(input_ids, train['label'].values, random_state=42, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks,
                                                       input_ids,
                                                       random_state=42,
                                                       test_size=0.1)

### Tensor Casting
- numpy ndarray로 되어있는 input, label, mask들을 torch tensor로 변환한다

In [25]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

### Batch 및 DataLoader 설정
- 현재 쓰고 있는 GPU의 VRAM에 맞도록 배치사이즈를 설정한다.
- !nvidia-smi
- 우선 배치사이즈를 크게 넣어보고 VRAM 부족 메시지(Out-of-Memory)가 나오면 8의 배수 중 더 작은 것으로 줄여나가는 것이 일반적인 방법

In [ ]:
BATCH_SIZE = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

### 테스트셋(Testset) 전처리
- 위의 train-val 셋 전처리와 동일
- train 전처리할 때 함께 하는 것이 생산성 측면에서 나음

In [27]:
sentences = test['document']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = test['label'].values

# 토크나이제이션
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 고유 정수 인코딩
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# 패딩하기
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

### 모델 학습
- GPU 체크 및 할당
  - GPU 8개가 있고 이 중 0번째 GPU TITAN X를 사용
  - CPU를 사용해도 학습 가능
- argparse
- ex) python a.py -batch 32 -epoch 100

In [28]:
# GPU 체크 및 할당
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    # print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead :(')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


#### 분류를 위한 BERT 모델 생성
- transformers의 BertForSequenceClassification 모듈을 이용
- 이진분류(binary classification) -> num_labels = 2
  - 만약에 label 수가 달라지면 변수만 고치면 됨!



In [29]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

### Learning Scheduler (학습 스케줄링)
- transformers에서 제공하는 옵티마이저 중 AdamW를 사용
- 훈련 스텝 => iteration * epoch

In [30]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,   # 학습률 (learning rate)
                  eps = 1e-8   # 0으로 나누는 것(division-by-zero)을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs

# scheduler 선언 -> lr을  천천히 감소시킴
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# 학습

### accuracy와 시간 표시함수 정의

In [32]:
# Helper Function

# 정확도(Acc) 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시(TimeStamp) 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

### 함수 실행 부분
- 데이터로더에서 배치만큼 가져온 후 forward, backward pass를 수행한다
- gradient update는 명시적으로 하지 않고 위에서 로드한 optimizer를 활용

In [ ]:
# !!!!!재현!!!!!을 위해 random seed 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 50 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑(gradient clipping)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)

    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

# =========================== validation ===========================
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # 그래디언트 계산 안함
        with torch.no_grad():
            # Forward 수행
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("Done!")


======== Epoch 1 / 4 ========
Training...
  Batch    50  of  4,219.    Elapsed: 0:00:33.
  Batch   100  of  4,219.    Elapsed: 0:01:05.
  Batch   150  of  4,219.    Elapsed: 0:01:38.
  Batch   200  of  4,219.    Elapsed: 0:02:10.
  Batch   250  of  4,219.    Elapsed: 0:02:42.
  Batch   300  of  4,219.    Elapsed: 0:03:15.
  Batch   350  of  4,219.    Elapsed: 0:03:47.
  Batch   400  of  4,219.    Elapsed: 0:04:20.
  Batch   450  of  4,219.    Elapsed: 0:04:52.
  Batch   500  of  4,219.    Elapsed: 0:05:25.
  Batch   550  of  4,219.    Elapsed: 0:05:57.
  Batch   600  of  4,219.    Elapsed: 0:06:30.
  Batch   650  of  4,219.    Elapsed: 0:07:02.
  Batch   700  of  4,219.    Elapsed: 0:07:35.
  Batch   750  of  4,219.    Elapsed: 0:08:07.
  Batch   800  of  4,219.    Elapsed: 0:08:40.
  Batch   850  of  4,219.    Elapsed: 0:09:12.
  Batch   900  of  4,219.    Elapsed: 0:09:45.
  Batch   950  of  4,219.    Elapsed: 0:10:17.
  Batch 1,000  of  4,219.    Elapsed: 0:10:50.
  Batch 1,050  of